# 1. Import

In [4]:
import pandas as pd
from pathlib import Path
import openpyxl
import string

import re
import numpy as np

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer

from scipy import sparse
import joblib

DATA_PATH = Path("../data/")

In [5]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('omw-1.4')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sahn1\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\sahn1\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sahn1\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\sahn1\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\sahn1\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

# 2. Load Data

In [6]:
headlines_df = pd.read_csv(DATA_PATH / "NVIDIA_NewsHeadlines_20231101-Present.csv")

stock_df = pd.read_excel(DATA_PATH / "NVIDIA_StockPriceDaily_20231101-Present.xlsx", engine='openpyxl', header=None)

# Now manually assign column names:
stock_df.columns = [
    'date', 'open', 'high', 'low', 'close', 'adj_close', 'volume'
]

stock_df["date"] = pd.to_datetime(stock_df["date"]).dt.date
headlines_df["date"] = pd.to_datetime(headlines_df["seendate"]).dt.date

In [7]:
stock_df.head()

,date,open,high,low,close,adj_close,volume
0,2025-11-21,181.24,184.56,172.93,178.88,178.88,346068500
1,2025-11-20,195.95,196.00,179.85,180.64,180.64,343504800
2,2025-11-19,184.79,187.86,182.83,186.52,186.52,247246400
3,2025-11-18,183.38,184.80,179.65,181.36,181.36,213598900
4,2025-11-17,185.97,189.00,184.32,186.60,186.60,172821100


In [8]:
headlines_df.head()

,title,seendate,url,sourcecountry,language,domain,date
0,5 Uncommon ETFs in Which to Invest - Insider M...,2023-11-01 00:00:00+00:00,https://www.insidermonkey.com/blog/5-uncommon-...,United States,English,insidermonkey.com,2023-11-01
1,Opinion : AMD makes bold prediction for its AI...,2023-11-01 01:30:00+00:00,https://www.marketwatch.com/story/amd-makes-bo...,United States,English,marketwatch.com,2023-11-01
2,Where Will Nvidia Be in 1 Year ? | The Motley ...,2023-11-01 10:00:00+00:00,https://www.fool.com/investing/2023/11/01/wher...,United States,English,fool.com,2023-11-01
3,"AMD forecasts $2 billion sales of AI chips , h...",2023-11-01 10:15:00+00:00,https://biztoc.com/x/2cd3129762787d99,NaN,English,biztoc.com,2023-11-01
4,AMD slides as muted outlook clouds Q3 earnings...,2023-11-01 17:45:00+00:00,https://www.myrtlebeachonline.com/news/busines...,United States,English,myrtlebeachonline.com,2023-11-01


In [9]:
merged_df = pd.merge(headlines_df, stock_df, on='date', how='inner')
desired_order = [
    'language',
    'sourcecountry',
    'seendate',
    'date',
    'url',
    'title',
    'domain',
    'open',
    'high',
    'low',
    'close',
    'adj_close',
    'volume'
]

merged_df = merged_df[desired_order]
merged_df['date'] = pd.to_datetime(merged_df['date'])

merged_df.head()

,language,sourcecountry,seendate,date,url,title,domain,open,high,low,close,adj_close,volume
0,English,United States,2023-11-01 00:00:00+00:00,2023-11-01,https://www.insidermonkey.com/blog/5-uncommon-...,5 Uncommon ETFs in Which to Invest - Insider M...,insidermonkey.com,40.88,42.38,40.87,42.33,42.3,437593000
1,English,United States,2023-11-01 01:30:00+00:00,2023-11-01,https://www.marketwatch.com/story/amd-makes-bo...,Opinion : AMD makes bold prediction for its AI...,marketwatch.com,40.88,42.38,40.87,42.33,42.3,437593000
2,English,United States,2023-11-01 10:00:00+00:00,2023-11-01,https://www.fool.com/investing/2023/11/01/wher...,Where Will Nvidia Be in 1 Year ? | The Motley ...,fool.com,40.88,42.38,40.87,42.33,42.3,437593000
3,English,NaN,2023-11-01 10:15:00+00:00,2023-11-01,https://biztoc.com/x/2cd3129762787d99,"AMD forecasts $2 billion sales of AI chips , h...",biztoc.com,40.88,42.38,40.87,42.33,42.3,437593000
4,English,United States,2023-11-01 17:45:00+00:00,2023-11-01,https://www.myrtlebeachonline.com/news/busines...,AMD slides as muted outlook clouds Q3 earnings...,myrtlebeachonline.com,40.88,42.38,40.87,42.33,42.3,437593000


In [10]:
merged_df.to_csv(DATA_PATH / "NVIDIA_Merged_20231101-Present.csv", index=False)

# 3. Split Data

In [11]:
# -------------------------------------------------------------------
# Train / test split by date (training = pre 2025-10-01)
# -------------------------------------------------------------------
SPLIT_DATE = pd.to_datetime('2025-10-01')

train_df = merged_df[merged_df['date'] < SPLIT_DATE].copy()
test_df  = merged_df[merged_df['date'] >= SPLIT_DATE].copy()

In [17]:
# find if there are any nas
train_df = train_df.dropna(subset=['title'])
test_df = test_df.dropna(subset=['title'])

# 4. Text Cleaning for TF IDF

In [20]:
# -------------------------------------------------------------------
# 2. Text cleaning
# -------------------------------------------------------------------

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words("english"))

# Add nvidia variants to custom stopwords
custom_stop = {"nvidia", "nvidias", "nvdia", "NVDA", "nvda", "nasdaq"}

def preprocess_text(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = re.sub(r'\d+', '', text)

    tokens = nltk.word_tokenize(text)

    tokens = [w for w in tokens if w not in stop_words and w not in custom_stop]
    tokens = [lemmatizer.lemmatize(w) for w in tokens]

    return ' '.join(tokens)

train_df["cleaned_title"] = [preprocess_text(t) for t in train_df["title"]]
test_df["cleaned_title"]  = [preprocess_text(t) for t in test_df["title"]]

In [21]:
train_daily = (
    train_df
    .groupby('date')['cleaned_title']
    .apply(lambda x: " ".join(x))
    .reset_index()
    .sort_values('date')
)

test_daily = (
    test_df
    .groupby('date')['cleaned_title']
    .apply(lambda x: " ".join(x))
    .reset_index()
    .sort_values('date')
)

# 5. TFIDF on Training data and Save

In [22]:
# -------------------------------------------------------------------
# 2) TF-IDF on TRAIN, then transform TEST with same vectorizer
# -------------------------------------------------------------------
vectorizer = TfidfVectorizer(
    max_features=50,      # you can tune this
    ngram_range=(1, 2),     # unigrams + bigrams
    min_df=2                # ignore super-rare tokens
)

X_train_text = vectorizer.fit_transform(train_daily['cleaned_title'])
X_test_text  = vectorizer.transform(test_daily['cleaned_title'])

In [23]:
# Save sparse TF-IDF matrices
sparse.save_npz(f"{DATA_PATH}/tfidf/X_train_tfidf.npz", X_train_text)
sparse.save_npz(f"{DATA_PATH}/tfidf/X_test_tfidf.npz", X_test_text)

# Save the vectorizer (vocabulary + settings)
joblib.dump(vectorizer, f"{DATA_PATH}/tfidf/tfidf_vectorizer.pkl")

['..\\data/tfidf/tfidf_vectorizer.pkl']